# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 

##  Final Project: Milestone 2 - Exploratory Data Analysis (EDA) [3 pts]


**Harvard University**<br/>
**Spring 2020**<br/>
**Group Members**: Fernando Medeiros, Mohammed Gufran Pathan, and Prerna Aggarwal<br/>

<hr style="height:2pt">

---

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### INSTRUCTIONS

- To submit your assignment follow the instructions given in Canvas.

<div class="theme"> Overview </div> 

As part of your final project,  please perform exploratory data analysis (EDA) on the data you have received. In order to simulate the "real world," the details of this are deliberately left open-ended, so your EDA should be what you consider the best for the domain you are working in and the specifics of the data. Please cite any references that you use when learning more about the topic of your project.

 

Key Questions: 

Given everything you have learned, if you faced this data set in the wild, how would you proceed? 

What are the important measures? 

What are the right questions to ask, and how can the data answer them?

In [2]:
import json
import lzma
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
f = lzma.open("./data/text/new_mexico.xz","rb")
data = f.readlines()
f.close()
data_json = [json.loads(line) for line in data]
data = json_normalize(data_json)

In [4]:
data.columns

Index(['id', 'url', 'name', 'name_abbreviation', 'decision_date',
       'docket_number', 'first_page', 'last_page', 'citations', 'frontend_url',
       'preview', 'volume.url', 'volume.barcode', 'volume.volume_number',
       'reporter.url', 'reporter.full_name', 'reporter.id', 'court.url',
       'court.name_abbreviation', 'court.slug', 'court.name', 'court.id',
       'jurisdiction.name', 'jurisdiction.name_long',
       'jurisdiction.whitelisted', 'jurisdiction.url', 'jurisdiction.id',
       'jurisdiction.slug', 'casebody.data.judges', 'casebody.data.attorneys',
       'casebody.data.opinions', 'casebody.data.head_matter',
       'casebody.data.corrections', 'casebody.status',
       'casebody.data.parties'],
      dtype='object')

In [5]:
data.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,frontend_url,...,jurisdiction.url,jurisdiction.id,jurisdiction.slug,casebody.data.judges,casebody.data.attorneys,casebody.data.opinions,casebody.data.head_matter,casebody.data.corrections,casebody.status,casebody.data.parties
0,17491,https://api.capapi.org/v1/cases/17491/,"STATE of New Mexico, Plaintiff-Appellee, v. Lo...",State v. Woodruff,1997-11-21,No. 23958,388,399,"[{'type': 'official', 'cite': '124 N.M. 388'},...",https://cite.capapi.org/nm/124/388/,...,https://api.capapi.org/v1/jurisdictions/nm/,52,nm,"[FRANCHINI, C.J., and BACA, SERNA and McKINNON...","[Tangora & Whitley, P.C., Robert E. Tangora, S...","[{'type': 'majority', 'text': 'OPINION MINZNER...",1997-NMSC-061\n951 P.2d 605\nSTATE of New Mexi...,,ok,NaN
1,17129,https://api.capapi.org/v1/cases/17129/,"STATE of New Mexico, ex rel. CHILDREN, YOUTH A...","State ex rel. Children, Youth & Families Depar...",1998-01-29,No. 17771,735,745,"[{'type': 'official', 'cite': '124 N.M. 735'},...",https://cite.capapi.org/nm/124/735/,...,https://api.capapi.org/v1/jurisdictions/nm/,52,nm,"[DONNELLY and BOSSON, JJ., concur.]","[Angela Adams, Chief Children’s Court Attorney...","[{'type': 'majority', 'text': 'OPINION WECHSLE...",1998-NMCA-039\n955 P.2d 204\nSTATE of New Mexi...,,ok,NaN
2,18630,https://api.capapi.org/v1/cases/18630/,"Caye C. BUCKINGHAM, Plaintiff-Appellant, v. Ja...",Buckingham v. Ryan,1997-12-17,"Nos. 17829, 17998",498,504,"[{'type': 'official', 'cite': '124 N.M. 498'},...",https://cite.capapi.org/nm/124/498/,...,https://api.capapi.org/v1/jurisdictions/nm/,52,nm,"[APODACA and ARMIJO, JJ., concur.]","[Maria Garcia Geer, Geer, Wissel & Levy, P.A.,...","[{'type': 'majority', 'text': 'OPINION PICKARD...",1998-NMCA-012\n953 P.2d 33\nCaye C. BUCKINGHAM...,,ok,NaN
3,18319,https://api.capapi.org/v1/cases/18319/,"STATE of New Mexico ex rel. Alan E. WHITEHEAD,...",State ex rel. Whitehead v. Vescovi-Dial,1997-10-23,No. 17715,375,380,"[{'type': 'official', 'cite': '124 N.M. 375'},...",https://cite.capapi.org/nm/124/375/,...,https://api.capapi.org/v1/jurisdictions/nm/,52,nm,"[PICKARD and FLORES, JJ., concur.]","[Tom Udall, Attorney General, Arthur W. Pepin,...","[{'type': 'majority', 'text': 'OPINION BOSSON,...",950 P.2d 818\nSTATE of New Mexico ex rel. Alan...,,ok,NaN
4,17318,https://api.capapi.org/v1/cases/17318/,"STATE of New Mexico, Plaintiff-Appellee, v. Sa...",State v. Rivera,1997-09-09,No. 17750,211,214,"[{'type': 'official', 'cite': '124 N.M. 211'},...",https://cite.capapi.org/nm/124/211/,...,https://api.capapi.org/v1/jurisdictions/nm/,52,nm,"[ALARID and BUSTAMANTE, JJ., concur.]","[Tom Udall, Attorney General, Elizabeth Blaisd...","[{'type': 'majority', 'text': 'OPINION PICKARD...",1997-NMCA-102\n947 P.2d 168\nSTATE of New Mexi...,,ok,NaN


In [6]:
data_json[1]

{'id': 17129,
 'url': 'https://api.capapi.org/v1/cases/17129/',
 'name': 'STATE of New Mexico, ex rel. CHILDREN, YOUTH AND FAMILIES DEPARTMENT, Petitioner-Appellee, In the Matter of ESPERANZA M., a child, and concerning Marian M., and Jesus M., Respondents-Appellants',
 'name_abbreviation': 'State ex rel. Children, Youth & Families Department',
 'decision_date': '1998-01-29',
 'docket_number': 'No. 17771',
 'first_page': '735',
 'last_page': '745',
 'citations': [{'type': 'official', 'cite': '124 N.M. 735'},
  {'type': 'parallel', 'cite': '1998-NMCA-039; 955 P.2d 204'}],
 'volume': {'url': 'https://api.capapi.org/v1/volumes/32044056060890/',
  'barcode': '32044056060890',
  'volume_number': '124'},
 'reporter': {'url': 'https://api.capapi.org/v1/reporters/554/',
  'full_name': 'New Mexico Reports',
  'id': 554},
 'court': {'url': 'https://api.capapi.org/v1/courts/nm-ct-app/',
  'name_abbreviation': 'N.M. Ct. App.',
  'slug': 'nm-ct-app',
  'name': 'Court of Appeals of New Mexico',
  'i

In [7]:
data['preview'][1]

[]

In [8]:
print(data['casebody.data.head_matter'][1])

1998-NMCA-039
955 P.2d 204
STATE of New Mexico, ex rel. CHILDREN, YOUTH AND FAMILIES DEPARTMENT, Petitioner-Appellee, In the Matter of ESPERANZA M., a child, and concerning Marian M., and Jesus M., Respondents-Appellants.
No. 17771.
Court of Appeals of New Mexico.
Jan. 29, 1998.
Angela Adams, Chief Children’s Court Attorney, Diane Garrity, Roy E. Stephenson, Jean Conner, Children’s Court Attorneys, Santa Fe, for Petitioner-Appellee.
Michael E. Vigil, Marchiondo, Vigil & Associates, P.C., Albuquerque, for Respondents-Appellants.
Jennifer Davis Hall, Albuquerque, Appellate Guardian ad litem.


In [9]:
json_normalize(data['casebody.data.opinions'][1])

,type,text,author
0,majority,"OPINION\nWECHSLER, Judge.\n1. As a result of a...","WECHSLER, Judge."


In [10]:
json_normalize(data['casebody.data.opinions'][1])['text']

0    OPINION\nWECHSLER, Judge.\n1. As a result of a...
Name: text, dtype: object

In [11]:
data['casebody.data.corrections'][1]

''

## List of EDAs
1. ~~Volume of cases by day/month (time trendline)~~ I'm not sure if this is relevant for our project (Fernando)
1. Histogram of number cases authored by each judge
1. Length of judgments (pages and word count)
1. No. of cases with a minority opinion
1. Bar graph of number of cases by court
1. Missing values **(misspellings?--Fernando)**
1. Word cloud
1. Visualize embeddings of the text
1. Size of vocabulary
1. No. of distinct words used per judgment (histogram)
1. Most common words used (excluding stop-words)
1. tf-idf of words (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
1. Categorical meta-data
1. Entity-recognition? (https://github.com/openlegaldata/legal-ner) (https://spacy.io/usage/linguistic-features#named-entities)
1. Sentiment of judgment?
